# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-18-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-18-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-06-19 04:22:33,33.93911,67.709953,98734,3934,63426.0,31374.0,Afghanistan,253.630125,3.984443
1,NaN,NaN,NaN,Albania,2021-06-19 04:22:33,41.15330,20.168300,132484,2454,129903.0,127.0,Albania,4603.655570,1.852299
2,NaN,NaN,NaN,Algeria,2021-06-19 04:22:33,28.03390,1.659600,135219,3615,94093.0,37511.0,Algeria,308.359826,2.673441
3,NaN,NaN,NaN,Andorra,2021-06-19 04:22:33,42.50630,1.521800,13842,127,13650.0,65.0,Andorra,17914.967967,0.917497
4,NaN,NaN,NaN,Angola,2021-06-19 04:22:33,-11.20270,17.873900,37467,853,31569.0,5045.0,Angola,113.998340,2.276670


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,84050,85892,87716,88740,89861,91458,93272,93288,96531,98734
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132415,132426,132437,132449,132459,132461,132469,132476,132481,132484
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,132034,132355,132727,133070,133388,133742,134115,134458,134840,135219


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,3305,3356,3412,3449,3527,3612,3683,3683,3842,3934
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2452,2452,2453,2453,2453,2453,2454,2454,2454,2454
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3544,3552,3560,3565,3571,3579,3588,3598,3605,3615


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,59871,60280,60598,60939,61410,61706,61820,62397,62698,63426
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,129761,129785,129807,129826,129842,129865,129879,129888,129895,129903
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,91894,92132,92384,92620,92852,93094,93355,93586,93831,94093


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7209,7211,7215,7215,7215,7229,7230,7230,7230,7241
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,21755,21765,21774,21774,21774,21805,21833,21839,21839,21868
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2346,2345,2345,2345,2345,2345,2346,2346,2346,2345


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,113,113,113,113,113,113,113,113,113,113
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,313,313,313,313,313,313,313,313,313,314
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,59,59,59,59,59,59,59,59,59,59


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
655,1001.0,Autauga,Alabama,US,2021-06-19 04:22:33,32.539527,-86.644082,7241,113,NaN,NaN,"Autauga, Alabama, US",12960.675867,1.560558
692,1075.0,Lamar,Alabama,US,2021-06-19 04:22:33,33.779950,-88.096680,1468,37,NaN,NaN,"Lamar, Alabama, US",10633.828323,2.520436
693,1077.0,Lauderdale,Alabama,US,2021-06-19 04:22:33,34.901719,-87.656247,9603,250,NaN,NaN,"Lauderdale, Alabama, US",10355.983565,2.603353


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,33516175,601571,0.0
India,29762793,383490,28580647.0
Brazil,17801462,498499,15687189.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,33516175,601571,0.0,32914604.0,2021-06-19 04:22:33,37.950547,-91.419547
India,29762793,383490,28580647.0,798656.0,2021-06-19 04:22:33,23.088275,81.806127
Brazil,17801462,498499,15687189.0,1615774.0,2021-06-19 04:22:33,-12.669522,-48.480493
France,5814648,110864,401836.0,5301948.0,2021-06-19 04:22:33,6.430808,-34.730285
Turkey,5359728,49071,5224224.0,86433.0,2021-06-19 04:22:33,38.963700,35.243300


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3806465,63274,0.0
Texas,2983636,52083,0.0
Florida,2341116,37555,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3806465,63274,0.0,3743191.0,2021-06-19 04:22:33,37.843962,-120.728594
Texas,2983636,52083,0.0,2931553.0,2021-06-19 04:22:33,31.660643,-98.653069
Florida,2341116,37555,0.0,2303561.0,2021-06-19 04:22:33,28.940755,-82.700744
New York,2111162,53597,0.0,2057565.0,2021-06-19 04:22:33,42.544151,-75.474183
Illinois,1388634,25566,0.0,1363068.0,2021-06-19 04:22:33,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1247033,24444,0.0
Illinois,Cook,556201,10461,0.0
Arizona,Maricopa,555891,10193,0.0
Florida,Miami-Dade,504650,6472,0.0
Texas,Harris,402916,6540,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1247033,24444,0.0,1222589.0,2021-06-19 04:22:33,34.308284,-118.228241,6037.0
Illinois,Cook,556201,10461,0.0,545740.0,2021-06-19 04:22:33,41.841448,-87.816588,17031.0
Arizona,Maricopa,555891,10193,0.0,545698.0,2021-06-19 04:22:33,33.348359,-112.491815,4013.0
Florida,Miami-Dade,504650,6472,0.0,498178.0,2021-06-19 04:22:33,25.611236,-80.551706,12086.0
Texas,Harris,402916,6540,0.0,396376.0,2021-06-19 04:22:33,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-16,93288,132476,134458,13836,37094,1263,4198620,223904,30301,649002,...,4605805,346515,104463,4,255457,11794,312164,6867,118850,40556
2021-06-17,96531,132481,134840,13839,37289,1263,4222400,224000,30309,649181,...,4616628,348662,104834,4,256862,12150,312334,6869,122244,40927
2021-06-18,98734,132484,135219,13842,37467,1263,4242763,224086,30322,649309,...,4626915,351378,105219,4,258140,12508,312499,6875,125157,41335


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-16,3683,2454,3598,127,847,42,87261,4490,910,10671,...,128190,5120,711,1,2886,61,3544,1351,1492,1640
2021-06-17,3842,2454,3605,127,851,42,87789,4491,910,10674,...,128209,5152,713,1,2906,61,3545,1351,1525,1647
2021-06-18,3934,2454,3615,127,853,42,88247,4492,910,10677,...,128220,5207,717,1,2925,62,3548,1352,1554,1656


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-16,62397,129888,93586,13633,31003,1214,3791979,215533,23716,634900,...,15563,309965,100328,3,235795,4590,304777,3781,100423,37075
2021-06-17,62698,129895,93831,13640,31105,1221,3818346,215655,23720,635192,...,15601,313388,100623,3,236983,4653,305205,3840,101885,37109
2021-06-18,63426,129903,94093,13650,31569,1221,3844329,215893,23727,635544,...,15607,317016,101018,3,238241,4733,305658,3846,103884,37143


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7209,7211,7215,7215,7215,7229,7230,7230,7230,7241
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1459,1458,1458,1458,1458,1462,1462,1465,1465,1468
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9570,9578,9582,9582,9582,9591,9592,9596,9596,9603
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3111,3118,3121,3121,3121,3124,3125,3129,3129,3133
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16211,16227,16234,16234,16234,16248,16259,16265,16265,16278


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-16,548323,70770,0,888005,344657,3805098,552183,348560,109418,49,...,865750,2981826,410377,24348,3729,678226,445686,163311,676442,61359
2021-06-17,548323,70770,0,888337,344945,3806154,552726,348595,109470,49,...,865750,2982619,410746,24357,3747,678392,446124,163382,676551,61425
2021-06-18,548657,70875,0,888701,345221,3806465,553200,348665,109501,49,...,866027,2983636,411071,24360,3752,678506,446682,163382,676673,61490


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-06-16        7230   21839    2346  2684   6944    1242   2240   14718   
2021-06-17        7230   21839    2346  2684   6944    1242   2240   14718   
2021-06-18        7241   21868    2345  2685   6945    1243   2242   14719   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-16         3726     1872  ...    2883    672     3249      802   
2021-06-17         3726     1872  ...    2886    673     3249      802   
2021-06-18         3728     1873  ...    2886    672     3250      802   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-16           4664  3797  2302          0      925    656  
2021-06-17           4671  3797  2301          0      925    656  
2021-06-18           4677  3797  2302          0      925    656  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-16,11288,373,0,17791,5867,63227,6709,8265,1679,0,...,12506,52007,2324,256,30,11330,5801,2861,8041,734
2021-06-17,11288,373,0,17809,5869,63256,6713,8266,1679,0,...,12506,52040,2324,256,30,11336,5810,2863,8046,734
2021-06-18,11306,373,0,17822,5874,63274,6723,8270,1679,0,...,12516,52083,2326,256,30,11342,5820,2863,8047,734


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-06-16         113     313      59   64    139      42     71     325   
2021-06-17         113     313      59   64    139      42     71     325   
2021-06-18         113     314      59   64    139      42     71     327   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-16          124       45  ...      30     11       31        7   
2021-06-17          124       45  ...      30     11       31        7   
2021-06-18          124       45  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-16             40    11    13          0       26      6  
2021-06-17             40    11    13          0       26      6  
2021-06-18             40    11    13          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-16,0.000172,0.000053,0.002558,0.000579,0.004686,0.0,0.006202,0.000442,0.000495,0.000236,...,0.001917,0.008440,0.003362,0.0,0.005277,0.043347,0.000692,0.000437,0.026126,0.005903
2021-06-17,0.034763,0.000038,0.002841,0.000217,0.005257,0.0,0.005664,0.000429,0.000264,0.000276,...,0.002350,0.006196,0.003551,0.0,0.005500,0.030185,0.000545,0.000291,0.028557,0.009148
2021-06-18,0.022822,0.000023,0.002811,0.000217,0.004774,0.0,0.004823,0.000384,0.000429,0.000197,...,0.002228,0.007790,0.003672,0.0,0.004975,0.029465,0.000528,0.000873,0.023829,0.009969


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-16,0.000000,0.0,0.002787,0.0,0.005938,0.0,0.007458,0.000223,0.0,0.000281,...,0.000070,0.006092,0.002821,0.0,0.007330,0.000000,0.000565,0.000741,0.033241,0.001833
2021-06-17,0.043171,0.0,0.001946,0.0,0.004723,0.0,0.006051,0.000223,0.0,0.000281,...,0.000148,0.006250,0.002813,0.0,0.006930,0.000000,0.000282,0.000000,0.022118,0.004268
2021-06-18,0.023946,0.0,0.002774,0.0,0.002350,0.0,0.005217,0.000223,0.0,0.000281,...,0.000086,0.010675,0.005610,0.0,0.006538,0.016393,0.000846,0.000740,0.019016,0.005464


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-16,0.009334,0.000069,0.002474,0.000514,0.007409,0.000000,0.005305,0.000362,0.000253,0.000727,...,0.000064,0.009037,0.003420,0.0,0.005981,0.011236,0.001271,0.001589,0.005286,0.000513
2021-06-17,0.004824,0.000054,0.002618,0.000513,0.003290,0.005766,0.006953,0.000566,0.000169,0.000460,...,0.002442,0.011043,0.002940,0.0,0.005038,0.013725,0.001404,0.015604,0.014558,0.000917
2021-06-18,0.011611,0.000062,0.002792,0.000733,0.014917,0.000000,0.006805,0.001104,0.000295,0.000554,...,0.000385,0.011577,0.003926,0.0,0.005308,0.017193,0.001484,0.001562,0.019620,0.000916


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-16,0.000292,0.000749,NaN,0.000491,0.000836,0.000195,0.000841,0.000112,-0.000027,0.0,...,0.000276,0.000717,0.001007,0.000370,0.003228,0.000409,0.001193,0.000551,0.000053,0.000913
2021-06-17,0.000000,0.000000,NaN,0.000374,0.000836,0.000278,0.000983,0.000100,0.000475,0.0,...,0.000000,0.000266,0.000899,0.000370,0.004827,0.000245,0.000983,0.000435,0.000161,0.001076
2021-06-18,0.000609,0.001484,NaN,0.000410,0.000800,0.000082,0.000858,0.000201,0.000283,0.0,...,0.000320,0.000341,0.000791,0.000123,0.001334,0.000168,0.001251,0.000000,0.000180,0.001058


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-16      0.000000  0.000275  0.000000  0.000746  0.000865  0.000000   
2021-06-17      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2021-06-18      0.001521  0.001328 -0.000426  0.000373  0.000144  0.000805   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-06-16      0.000000  0.000272  0.000537  0.000000  ...  0.000347   
2021-06-17      0.000000  0.000000  0.000000  0.000000  ...  0.001041   
2021-06-18      0.000893  0.000068  0.000537  0.000534  ...  0.000000   

Province_State                                                              \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton     Uinta   
2021-06-16      0.001490  0.000000      0.0   0.001718 -0.000263  0.000000   
2021-06-17      0.001488  0.000000      0.0   0.001501  0.000000 -0.000434   
2021-06-18     -0.001486  0.000308      0.0   0.001285  0.000000  0.000435   

Province_State                             
Admin2         Unassigned Washakie Weston  
2021-06-16            NaN      0.0    0.0  
2021-06-17            NaN      0.0    0.0  
2021-06-18            NaN      0.0    0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-16,0.000621,0.0,NaN,0.000675,-0.000341,0.000301,0.001343,0.000242,0.0,NaN,...,0.00016,0.000770,0.000430,0.0,0.034483,0.000177,0.000172,0.001751,0.002244,0.0
2021-06-17,0.000000,0.0,NaN,0.001012,0.000341,0.000459,0.000596,0.000121,0.0,NaN,...,0.00000,0.000635,0.000000,0.0,0.000000,0.000530,0.001551,0.000699,0.000622,0.0
2021-06-18,0.001595,0.0,NaN,0.000730,0.000852,0.000285,0.001490,0.000484,0.0,NaN,...,0.00080,0.000826,0.000861,0.0,0.000000,0.000529,0.001721,0.000000,0.000124,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2021-06-16         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   
2021-06-17         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   
2021-06-18         0.0  0.003195     0.0  0.0    0.0     0.0    0.0  0.006154   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-16      0.00813      0.0  ...     0.0    0.0      0.0      0.0   
2021-06-17      0.00000      0.0  ...     0.0    0.0      0.0      0.0   
2021-06-18      0.00000      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-16            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-17            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-18            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-16,0.009077,0.000054,0.002622,0.000465,0.003956,1.520717e-07,0.005939,0.000371,0.000451,0.000229,...,0.001783,0.008119,0.003127,2.312965e-18,0.005127,0.037785,0.000735,0.000387,0.022614,0.005227
2021-06-17,0.021920,0.000046,0.002732,0.000341,0.004606,7.603585e-08,0.005801,0.000400,0.000357,0.000252,...,0.002066,0.007157,0.003339,1.156482e-18,0.005313,0.033985,0.000640,0.000339,0.025586,0.007187
2021-06-18,0.022371,0.000034,0.002771,0.000279,0.004690,3.801793e-08,0.005312,0.000392,0.000393,0.000225,...,0.002147,0.007474,0.003506,5.782412e-19,0.005144,0.031725,0.000584,0.000606,0.024708,0.008578


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-16,0.010189,0.000109,0.002522,2.349914e-16,0.006369,1.306365e-10,0.007049,0.000339,1.490927e-23,0.000352,...,0.000067,0.007832,0.002150,0.0,0.006831,0.010758,0.000877,0.000734,0.025749,0.001581
2021-06-17,0.026680,0.000054,0.002234,1.174957e-16,0.005546,6.531826e-11,0.006550,0.000281,7.454635e-24,0.000316,...,0.000108,0.007041,0.002482,0.0,0.006881,0.005379,0.000580,0.000367,0.023933,0.002924
2021-06-18,0.025313,0.000027,0.002504,5.874786e-17,0.003948,3.265913e-11,0.005884,0.000252,3.727318e-24,0.000299,...,0.000097,0.008858,0.004046,0.0,0.006709,0.010886,0.000713,0.000554,0.021475,0.004194


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-16,0.006575,0.000102,0.002583,0.000678,0.007939,1.501543e-07,0.006036,0.000524,0.000457,0.000607,...,0.000153,0.010251,0.003068,5.551115e-17,0.005950,0.034663,0.001175,0.002858,0.007736,0.000605
2021-06-17,0.005699,0.000078,0.002601,0.000596,0.005615,2.883106e-03,0.006494,0.000545,0.000313,0.000533,...,0.001297,0.010647,0.003004,2.775558e-17,0.005494,0.024194,0.001289,0.009231,0.011147,0.000761
2021-06-18,0.008655,0.000070,0.002696,0.000664,0.010266,1.441553e-03,0.006650,0.000824,0.000304,0.000544,...,0.000841,0.011112,0.003465,1.387779e-17,0.005401,0.020694,0.001387,0.005397,0.015384,0.000839


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-16,0.000418,0.000509,NaN,0.000479,0.000744,0.000242,0.000783,0.000142,0.000145,9.147746e-139,...,0.000269,0.000552,0.000779,0.000335,0.003284,0.000288,0.001092,0.000571,0.000118,0.001030
2021-06-17,0.000209,0.000254,NaN,0.000427,0.000790,0.000260,0.000883,0.000121,0.000310,4.573873e-139,...,0.000134,0.000409,0.000839,0.000352,0.004056,0.000266,0.001037,0.000503,0.000140,0.001053
2021-06-18,0.000409,0.000869,NaN,0.000418,0.000795,0.000171,0.000870,0.000161,0.000297,2.286937e-139,...,0.000227,0.000375,0.000815,0.000238,0.002695,0.000217,0.001144,0.000251,0.000160,0.001056


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-16      0.000293  0.000652  0.000107  0.001092  0.000698  0.000009   
2021-06-17      0.000146  0.000326  0.000054  0.000546  0.000349  0.000004   
2021-06-18      0.000834  0.000827 -0.000186  0.000459  0.000247  0.000405   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-06-16      0.000586  0.000363  0.000621  0.000012  ...  0.000363   
2021-06-17      0.000293  0.000181  0.000310  0.000006  ...  0.000702   
2021-06-18      0.000593  0.000125  0.000424  0.000270  ...  0.000351   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-06-16      0.001195  0.000218  0.000190   0.001526  0.000106  0.000300   
2021-06-17      0.001341  0.000109  0.000095   0.001513  0.000053 -0.000067   
2021-06-18     -0.000072  0.000208  0.000048   0.001399  0.000027  0.000184   

Province_State                                     
Admin2         Unassigned  Washakie        Weston  
2021-06-16           -1.0  0.000265  3.007039e-06  
2021-06-17           -1.0  0.000133  1.503520e-06  
2021-06-18           -1.0  0.000066  7.517598e-07  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-16,0.000588,0.000169,NaN,0.000566,0.000158,0.000221,0.001182,0.000158,0.000010,NaN,...,0.000207,0.000599,0.00028,4.787220e-07,0.026171,0.000566,0.000187,0.001362,0.001439,0.003117
2021-06-17,0.000294,0.000085,NaN,0.000789,0.000249,0.000340,0.000889,0.000140,0.000005,NaN,...,0.000103,0.000617,0.00014,2.393610e-07,0.013086,0.000548,0.000869,0.001030,0.001030,0.001559
2021-06-18,0.000944,0.000042,NaN,0.000760,0.000551,0.000312,0.001189,0.000312,0.000003,NaN,...,0.000451,0.000721,0.00050,1.196805e-07,0.006543,0.000539,0.001295,0.000515,0.000577,0.000779


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                      \
Admin2           Autauga   Baldwin       Barbour          Bibb        Blount   
2021-06-16      0.000010  0.000013  1.657546e-08  7.232780e-15  8.539462e-13   
2021-06-17      0.000005  0.000006  8.287728e-09  3.616390e-15  4.269731e-13   
2021-06-18      0.000002  0.001601  4.143864e-09  1.808195e-15  2.134865e-13   

Province_State                                                      \
Admin2               Bullock        Butler       Calhoun  Chambers   
2021-06-16      2.839576e-12  7.027918e-09  1.917733e-07  0.003939   
2021-06-17      1.419788e-12  3.513959e-09  9.588665e-08  0.001970   
2021-06-18      7.098939e-13  1.756980e-09  3.076971e-03  0.000985   

Province_State                ...       Wyoming                              \
Admin2              Cherokee  ...          Park        Platte      Sheridan   
2021-06-16      7.892986e-20  ...  4.457379e-28  1.809233e-44  3.366491e-30   
2021-06-17      3.946493e-20  ...  2.228689e-28  9.046165e-45  1.683246e-30   
2021-06-18      1.973247e-20  ...  1.114345e-28  4.523083e-45  8.416228e-31   

Province_State                                                              \
Admin2              Sublette Sweetwater     Teton         Uinta Unassigned   
2021-06-16      1.478930e-50   0.006410  0.000868  4.736952e-15   0.752941   
2021-06-17      7.394650e-51   0.003205  0.000434  2.368476e-15   0.752941   
2021-06-18      3.697325e-51   0.001603  0.000217  1.184238e-15   0.752941   

Province_State                          
Admin2              Washakie    Weston  
2021-06-16      9.188840e-43  0.000781  
2021-06-17      4.594420e-43  0.000391  
2021-06-18      2.297210e-43  0.000195  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
